## Install & Imports

In [1]:
!pip install -q ultralytics

In [2]:
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from collections import defaultdict

## Image Inference

In [8]:
import cv2
import numpy as np
import os
from glob import glob
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict

model = YOLO("model.pt")
names = model.model.names

input_images_folder = "images/"
output_results_folder = "results/"
image_files = glob(os.path.join(input_images_folder, "*.jpg"))

# Ensure the output folder exists
os.makedirs(output_results_folder, exist_ok=True)

# Initialize variables
confidence_threshold = 0.7
coin_value_mapping = {0: 0.50, 1: 0.25, 2: 1, 3: 0.10, 4: 10, 5: 20, 6: 0.05, 7: 5, 8: 50}

# Process each image in the folder
for image_file in image_files:

    coin_counts = defaultdict(int)

    image = cv2.imread(image_file)
    results = model(image)
    boxes = results[0].boxes.xyxy.cpu()

    # Annotator Init
    annotator = Annotator(image, line_width=2)

    # Process detected objects
    for box, cls, conf in zip(boxes, results[0].boxes.cls.cpu().tolist(), results[0].boxes.conf.float().cpu().tolist()):
        # Filter out predictions below the confidence threshold
        if conf >= confidence_threshold:
            annotator.box_label(box, color=colors(int(cls), True), label=f"{names[int(cls)]} {conf:.2f}")
            coin_counts[int(cls)] += 1

    # Calculate total value and update counter
    counter = len(boxes)
    total_value = sum(coin_value_mapping[coin_type] * count for coin_type, count in coin_counts.items())

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_size = 1
    font_thickness = 2

    cv2.putText(image, f'Number of Objects: {counter}', (10, 50), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)
    cv2.putText(image, f'Total Value: {total_value:.2f} JD', (10, 20), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)

    # Save the result image in the output folder
    result_image_path = os.path.join(output_results_folder, os.path.basename(image_file))
    cv2.imwrite(result_image_path, image)



0: 640x640 2 1-4-dinars, 1 10-Piastres, 2 10-dinars, 1 20-dinar, 2 5-Piastress, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 1-2-dinar, 2 1-dinars, 3 10-Piastress, 1 10-dinar, 1 5-Piastres, 2 5-dinars, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 640x640 1 1-2-dinar, 1 1-4-dinar, 2 1-dinars, 2 50-dinars, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 20-dinar, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


## Video Inference

In [20]:
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict
import time

track_history = defaultdict(lambda: [])
model = YOLO("model.pt")
names = model.model.names

video_path = "videos/Jordanian_dinar_coins_and_banknotes.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_video_path=video_path.split("/")[-1]
result = cv2.VideoWriter(f"results/{output_video_path}",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       30,  # Set a default FPS value (you can change it if needed)
                       (w, h))

font = cv2.FONT_HERSHEY_SIMPLEX
counter = 0
coin_counts = defaultdict(int)  # Initialize the count of each coin type

coin_value_mapping = {0: 0.50, 1: 0.25, 2: 1, 3: 0.10, 4: 10, 5: 20, 6: 0.05, 7: 5, 8: 50}  # Example mapping, adjust as needed
confidence_threshold = 0.75

# Initialize start_time here
start_time = time.time()

# Process the video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Track objects using YOLO model
        results = model.track(frame, conf=confidence_threshold, verbose=False, tracker="botsort.yaml")

        if results and results[0].boxes.id is not None:
            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, conf, track_id in zip(results[0].boxes.xyxy.cpu(), clss, confs, track_ids):
                # Filter out predictions below the confidence threshold
                if conf >= confidence_threshold:
                    annotator.box_label(box, color=colors(int(cls), True),
                                        label=f"{names[int(cls)]} {conf:.2f} ID:{track_id}")

                    # Store tracking history
                    track = track_history[track_id]
                    track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                    if len(track) > 30:
                        track.pop(0)

                    # Plot tracks
                    points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                    cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                    # cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

                    # Update coin counts only if the track is new
                    if len(track) == 1:
                        coin_counts[int(cls)] += 1

            # Calculate and display counter based on unique track IDs
            counter = len(set(track_ids))

        # Calculate and display dynamic FPS
        elapsed_time = time.time() - start_time
        fps = int(1 / elapsed_time)
        cv2.putText(frame, f'FPS: {fps}', (10, 110), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Calculate total value after processing all frames
        total_value = sum(coin_value_mapping[coin_type] * count for coin_type, count in coin_counts.items())
        cv2.putText(frame, f'Number of Objects: {counter}', (10, 50), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)
        cv2.putText(frame, f'Total Value: {total_value:.2f} JD', (10, 20), font, font_size, (255, 0, 0), font_thickness, cv2.LINE_AA)

        result.write(frame)
        start_time = time.time()

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()